In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
import numpy as np

In [ ]:
data_path = '/Users/jk1/temp/treatment_effects/prepro_test/gsu_Extraction_20220815_prepro_25112023_213851/preprocessed_features_25112023_213851.csv'
outcomes_path = '/Users/jk1/temp/treatment_effects/prepro_test/gsu_Extraction_20220815_prepro_25112023_213851/preprocessed_outcomes_final_25112023_213851.csv'

In [ ]:
data_df = pd.read_csv(data_path)
outcomes_df = pd.read_csv(outcomes_path)

In [ ]:
data_df.head()

In [ ]:
outcomes_df.head()

In [ ]:
pa_id = np.random.choice(data_df['case_admission_id'].unique())
pa_id

In [ ]:
vital_name = 'Glasgow Coma Scale'
temp = data_df[(data_df['case_admission_id'] == pa_id)
                                          & (data_df.sample_label.isin([vital_name]))].copy()
ax = sns.scatterplot(x='relative_sample_date_hourly_cat', y='value', data=temp, hue='value', legend=False)
ax.set_xlabel('Hours from admission')
ax.set_ylabel(vital_name)
ax.tick_params(axis="x", rotation=45)
ax.set_ylim(-3.2, 3.2)
ax.set_title(vital_name)

plt.show()

In [ ]:
data_df[(data_df['case_admission_id'] == pa_id) & (data_df.sample_label.isin(["cholesterol HDL"]))]

In [ ]:
data_df.source.unique()

In [ ]:
sample_labels = data_df.sample_label.unique()
sample_labels

In [ ]:

from matplotlib.lines import Line2D


def plot_features(pa_id, subject_df, sample_labels, outcome, outcome_name, plot_source=False):
    plt.subplots_adjust(hspace=0.2)

    # set number of columns (use 3 to demonstrate the change)
    ncols = 5
    # calculate number of rows
    nrows = len(sample_labels) // ncols + (len(sample_labels) % ncols > 0)
    plt.figure(figsize=(5*ncols, 5*nrows))

    for n, sample_label in enumerate(sample_labels):
    # add a new subplot iteratively using nrows and cols
        ax = plt.subplot(nrows, ncols, n + 1)
        label_df = subject_df[(subject_df.sample_label.isin([sample_label]))].copy()

        if not plot_source:
            ax = sns.scatterplot(x='relative_sample_date_hourly_cat', y='value', data=label_df, hue='value', legend=False, ax=ax)
        else:
            palette = {"EHR": "navy", "EHR_locf_imputed": "lavender", "EHR_pop_imputed": "magenta", "EHR_pop_imputed_locf_imputed": "pink",
                       "stroke_registry": "darkgreen", "stroke_registry_locf_imputed": "paleturquoise", "stroke_registry_pop_imputed": "magenta", "stroke_registry_pop_imputed_locf_imputed": "pink",
                       "notes": "darkgreen", "notes_locf_imputed": "paleturquoise", "notes_pop_imputed": "magenta", "notes_pop_imputed_locf_imputed": "pink",
                       'EHR_prespecified_imputed': 'orange', 'EHR_prespecified_imputed_prespecified_imputed': 'orange', 'EHR_prespecified_imputed_locf_imputed': 'bisque'}
            ax = sns.scatterplot(x='relative_sample_date_hourly_cat', y='value', data=label_df, hue='source', legend=False, ax=ax, palette=palette)

            if n == len(sample_labels) - 1:
                legend_elements = [Line2D([0], [0], marker='o', color='w', label=source,
                          markerfacecolor=palette[source], markersize=10) for source in palette.keys()]
                ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5))

        # chart formatting
        # ax.set_xlabel('Hours from admission')
        ax.tick_params(axis="x", rotation=45)
        if sample_label != 'anti_hypertensive_strategy':
            ax.set_ylim(-3, 3)
        ax.set_title(sample_label.upper())
        # ax.get_legend().remove()
        ax.set_xlabel("")

    fig = plt.gcf()
    fig.suptitle(f'{pa_id}; {outcome_name}: {outcome}', y=0.89, fontsize = 25)
    return fig


In [ ]:
subject_df = data_df[(data_df['case_admission_id'] == pa_id)]
subject_outcome = outcomes_df[outcomes_df.case_admission_id == pa_id]['3M Death'].values[0]

In [ ]:
plot_features(pa_id, subject_df, sample_labels, subject_outcome, '3M Death', plot_source=True)

In [ ]:
from preprocessing_tools.utils import ensure_dir

out_dir = os.path.join(os.path.dirname(data_path), 'data_visualisation')
ensure_dir(out_dir)

# Create data visualisations for all patients

In [ ]:
outcome = '3M Death'

In [ ]:
for pa_id in tqdm(data_df.case_admission_id.unique()):
    subject_df = data_df[(data_df['case_admission_id'] == pa_id)]
    outcome_values = outcomes_df[outcomes_df.case_admission_id == pa_id][outcome].values
    if len(outcome_values) > 0:
        subj_outcome = outcome_values[0]
    else:
        subj_outcome = np.nan
    fig = plot_features(pa_id, subject_df, sample_labels, subj_outcome, outcome, plot_source=True)
    fig.savefig(os.path.join(out_dir, pa_id + '.pdf'), bbox_inches='tight')
    plt.close()